# CSE252D: Homework 3
## Computing Resources
Please read the README file of this repository for the instructions
## Instructions
1. Attempt all questions.
2. Please comment all your code adequately.
3. Include all relevant information such as text answers, output images in notebook.
4. **Academic integrity:** The homework must be completed individually.

5. **Submission instructions:**  
 (a) Submit the notebook and its PDF version on Gradescope.  
 (b) Rename your submission files as Lastname_Firstname.ipynb and Lastname_Firstname.pdf.  
 (c) Correctly select pages for each answer on Gradescope to allow proper grading.

6. **Due date:** Sun, Jun 6, at 11:59pm.

## Q1: UNet for Image Segmentation

1. **Check the codes in `Segmentation`.** In this homework, we will provide the dataset loader, the evaluation code, the basic UNet structure and some useful functions. You will be asked to try different variations of network structure and decide the best training strategies to obtain good results. Like in previous homeworks, you are welcome to cite any open source codes that help you improve performance. The provided codes include:
    1. `test.py`: The file for evaluation. 
    2. `dataLoader.py`: The file to load the data for training and testing.  
    3. `model.py`: The file for models. The residual block (`ResBlock`) and the code to load pretrained weights of `resnet18  loadPretrainedWeight`) are given. The basic encoder and decoder are also given as a reference. 
    4. `colormap.mat`: The color map used to visualize segmentation results. 
    5. `utils.py`: The file for two useful functions. The `computeAccuracy` function computes the unnormalized confusion matrix of each batch of labels. The `save_label` function turns the label into an image using the given color map and saves the image at the assigned location. Also see `test.py` for how these two functions are being used. 
    6. `train.py`: An empty file where you will implement your training script. 

2. **Implement the network structures.**  You are required to implement 2 versions of UNet structure since the basic structure has already been given. In all three versions,  the `resnet18` structure before average pooling and fully connected layer will be used as the building block for encoder. You are strongly recommended to use weights pretrained on ImageNet, which may have a major impact on the performance. 
    1. `Basic UNet`: The code is given as a reference. Please see `encoder` and `decoder` class in `model.py`. The `encoder` comes from `resnet18` and the decoder consists of transpose convolutional layers and bilinear interpolation layers so that the final output will be of the same size as the image. Skip links are added to help the network recover more details. Please do not change the encoder. However, you are free to change the decoder, while ensuring that the structure of your decoder across three versions of the networks are similar so that you can make a fair comparison of their performances. 
    2. `UNet with dilation`: We modify the encoder to a dilated `resnet18` as described in Section 2 of [1] (You are not required to consider degridding in Section 4 of [1] ). We set the stride of the last 4 residual blocks to be 1 so that the highest level feature maps will be $4\times 4$ times larger. To increase the receptive field, we set the dilation of residual blocks that are fourth and third from the end to be 2, while the dilation of the residual blocks that are first and second from the end are set to 4.  The decoder should be modified accordingly. Implement your new encoder and decoder under class `encoderDilation` and `decoderDilation`. Ensure that for images of arbitrary shapes, the decoder will give segmentation outputs of the same shape.  **[15 points]**
    3. `UNet with dilation and pyramid pooling`:  Based on the encoder-decoder structure with dilation, add pyramid pooling layer after the last residual block of encoder.  Implement the pyramid pooling layer following [2]. Notice that after adding the pyramid layer, the number of channels of the output feature to the first transpose convolutional layer will change from 512 to 1024. Please implement your new encoder and decoder under classes `encoderSPP` and `decoderSPP`, respectively.  **[15 points]**

``Answer Q1.2.B here, Paste your code here``
```python
class encoderDilation(nn.Module):
    def __init__(self):

        # IMPLEMENT YOUR CODE HERE

        super(encoderDilation, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7,
                               stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.b1_1 = ResBlock(64, 64, 1)
        self.b1_2 = ResBlock(64, 64, 1)

        self.b2_1 = ResBlock(64, 128, 2)
        self.b2_2 = ResBlock(128, 128, 1)

        self.b3_1 = ResBlock(128, 256, 1, 2)
        self.b3_2 = ResBlock(256, 256, 1, 2)

        self.b4_1 = ResBlock(256, 512, 1, 4)
        self.b4_2 = ResBlock(512, 512, 1, 4)

    def forward(self, im):

        # IMPLEMENT YOUR CODE HERE

        x1 = F.relu(self.bn1(self.conv1(im)), inplace=True)
        x2 = self.b1_2(self.b1_1(self.maxpool(x1)))
        x3 = self.b2_2(self.b2_1(x2))
        x4 = self.b3_2(self.b3_1(x3))
        x5 = self.b4_2(self.b4_1(x4))
        return x1, x2, x3, x4, x5


class decoderDilation(nn.Module):
    def __init__(self, isSpp=False):
        
        # IMPLEMENT YOUR CODE HERE

        super(decoderDilation, self).__init__()
        self.conv1 = nn.Conv2d(512+256+128, 512, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(512)
        self.conv1_1 = nn.Conv2d(512, 21, 3, 1, 1, bias=False)
        self.bn1_1 = nn.BatchNorm2d(21)
        self.conv2 = nn.Conv2d(64+21, 21, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(21)
        self.conv3 = nn.Conv2d(21, 21, 3, 1, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(21)
        self.conv4 = nn.Conv2d(21, 21, 3, 1, 1, bias=False)
        self.sf = nn.Softmax(dim=1)

    def forward(self, im, x1, x2, x3, x4, x5):

        # IMPLEMENT YOUR CODE HERE

        _, _, nh, nw = x3.size()
        x5 = F.interpolate(x5, [nh, nw], mode='bilinear')
        x4 = F.interpolate(x4, [nh, nw], mode='bilinear')
        y1 = F.relu(self.bn1(self.conv1(torch.cat([x3, x4, x5], dim=1))), inplace=True)
        y1 = F.relu(self.bn1_1(self.conv1_1(y1)), inplace=True)

        _, _, nh, nw = x2.size()
        y1 = F.interpolate(y1, [nh, nw], mode='bilinear')
        y1 = torch.cat([y1, x2], dim=1)
        y2 = F.relu(self.bn2(self.conv2(y1)), inplace=True)

        _, _, nh, nw = x1.size()
        y2 = F.interpolate(y2, [nh, nw], mode='bilinear')
        y3 = F.relu(self.bn3(self.conv3(y2)), inplace=True)

        y4 = self.sf(self.conv4(y3))

        _, _, nh, nw = im.size()
        y4 = F.interpolate(y4, [nh, nw], mode='bilinear')

        p = -torch.log(torch.clamp(y4, min=1e-8))

        return p
```

``Answer Q1.2.C here, Paste your code here``

```python
class convolve_bn(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, bn=False):
        super(convolve_bn, self).__init__()

        bias = not bn
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation, bias=bias)
        self.bn = nn.BatchNorm2d(out_channels) if bn else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        return x


class SPP(nn.Module):
    def __init__(self, in_channels):

        super(SPP, self).__init__()
        size = [(1, 1), (2, 2), (3, 3), (6, 6)]

        self.pools = nn.ModuleList()
        for _, s in enumerate(size):
            pool_module = nn.Sequential(
                nn.AdaptiveAvgPool2d(output_size=s),
                convolve_bn(in_channels=in_channels, out_channels=in_channels // 4, kernel_size=1, bn=True)
            )
            self.pools.append(pool_module)

    def forward(self, x, nh, nw):
        
        out_list = [F.interpolate(x, [nh, nw], mode='bilinear')]
        
        for pool_module in self.pools:
            out = pool_module(x)
            out = F.interpolate(out, [nh, nw], mode='bilinear')
            out_list.append(out)

        output = torch.cat(out_list, dim=1)
        return output


class encoderSPP(nn.Module):
    def __init__(self):

        # IMPLEMENT YOUR CODE HERE

        super(encoderSPP, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7,
                               stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.b1_1 = ResBlock(64, 64, 1)
        self.b1_2 = ResBlock(64, 64, 1)

        self.b2_1 = ResBlock(64, 128, 2)
        self.b2_2 = ResBlock(128, 128, 1)

        self.b3_1 = ResBlock(128, 256, 1, 2)
        self.b3_2 = ResBlock(256, 256, 1, 2)

        self.b4_1 = ResBlock(256, 512, 1, 4)
        self.b4_2 = ResBlock(512, 512, 1, 4)

    def forward(self, im):

        # IMPLEMENT YOUR CODE HERE

        x1 = F.relu(self.bn1(self.conv1(im)), inplace=True)
        x2 = self.b1_2(self.b1_1(self.maxpool(x1)))
        x3 = self.b2_2(self.b2_1(x2))
        x4 = self.b3_2(self.b3_1(x3))
        x5 = self.b4_2(self.b4_1(x4))
        return x1, x2, x3, x4, x5


class decoderSPP(nn.Module):
    def __init__(self):
        
        super(decoderSPP, self).__init__()
        self.spp = SPP(in_channels=512)

        self.conv1 = nn.Conv2d(1024 + 256 + 128, 512, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(512)
        self.conv1_1 = nn.Conv2d(512, 21, 3, 1, 1, bias=False)
        self.bn1_1 = nn.BatchNorm2d(21)
        self.conv2 = nn.Conv2d(64 + 21, 21, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(21)
        self.conv3 = nn.Conv2d(21, 21, 3, 1, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(21)
        self.conv4 = nn.Conv2d(21, 21, 3, 1, 1, bias=False)
        self.sf = nn.Softmax(dim=1)

    def forward(self, im, x1, x2, x3, x4, x5):
        
        _, _, nh, nw = x3.size()
        x5 = self.spp(x5, nh, nw)

        x4 = F.interpolate(x4, [nh, nw], mode='bilinear')
        y1 = F.relu(self.bn1(self.conv1(torch.cat([x3, x4, x5], dim=1))), inplace=True)
        y1 = F.relu(self.bn1_1(self.conv1_1(y1)), inplace=True)

        _, _, nh, nw = x2.size()
        y1 = F.interpolate(y1, [nh, nw], mode='bilinear')
        y1 = torch.cat([y1, x2], dim=1)
        y2 = F.relu(self.bn2(self.conv2(y1)), inplace=True)

        _, _, nh, nw = x1.size()
        y2 = F.interpolate(y2, [nh, nw], mode='bilinear')
        y3 = F.relu(self.bn3(self.conv3(y2)), inplace=True)

        y4 = self.sf(self.conv4(y3))

        _, _, nh, nw = im.size()
        y4 = F.interpolate(y4, [nh, nw], mode='bilinear')

        p = -torch.log(torch.clamp(y4, min=1e-8))

        return p
```

3. **Implement training script and train the network.** Train your network using 1464 images from the training set of PASCAL VOC 2012. The dataset is on the server `/datasets/cs252-sp21-A00-public/hw3_data/VOCdevkit`. If you are not familiar with training scripts, you can refer to `test.py`  in this homework and `casia_train.py` in the previous homework. The structures of the training script are very similar. Please remember to output the training loss and training accuracy which may help you find the best hyper parameters.  **[40 points]**
    1. To accelerate the training speed, you can use the Pytorch multi-threaded data loader. **Important:** if you use multi-threaded data loader, remember to either randomly shuffle the data or change the random seeds after every epoch. Otherwise you will have severe overfitting issues because the data loader will always crop the same region of the image. 
    2. It is recommended to compute the prediction mIoU every epoch, since the curve of mIoU can be very different from the inverse of loss function. It may help you find the best training strategy. 
    3. To overcome over-fitting issues, you are encouraged to adopt more aggressive data augmentation methods, such as flipping the images or changing the intensity. 
    4. There are many things that may influence performance, such as learning rate, batch size and network structure of encoder and decoder. It might be hard to achieve state-of-the-art results. **The grading of the homework will not focus on the final mean IoU but more on analysis.** So don't be too worried if you cannot get a very good performance. Just make sure that you describe what you observe and answer the questions succinctly. 

``Point out where you implement your training script here``

The training script is implemented in `./Segmentation/train.py`. Or you can find it in the forked [repository](https://github.com/YaosenLin/cse252d-sp21-hw3)

4. **Answer the following questions:**
    1. Describe the loss function you use to train the semantic segmentation network. If you change the structure of the decoder, describe your new network architecture. **[10 points]**
    2. Describe your training details, such as: what kind of optimizer is used to train the network, what's the learning rate and the batch size, whether you decrease the learning rate as the training progresses, number of epochs required to train the network, or any other details that you find important for  performance. Note that in order to compare the three network structures, learning details for them should be the same. **[10 points]**
    3. Draw the loss curves of the three network structures in the same graph.  **[10 points]**
    4. Evaluate the trained models using the following commands. Draw a table to summarize quantitative performances of the 3 variations of the UNet structure. The table should include the IoU for each of the 21 categories of objects and the mean IoU across all categories. **[10 points]** :
        1. `Basic UNet`: `python test.py`. The testing mean IoU of 21 categories of object are stored in  `test/accuracy.npy`. You can add flags if necessary. 
        2. `UNet with dilation`: `python test.py --isDilation`. The testing mean IoU of 21 categories of objects are stored in `test_dilation/accuracy.npy`. You can add flags if necessary. 
        3. `UNet with dilation and pyramid pooling`: `python test.py --isSpp`. The testing mean IoU of 21 categories of object are stored in  `test_spp/accuracy.npy`. You can add flags if necessary. 
    5. Make a figure for qualitative comparisons of the 3 methods, shown on 4 different input images. Please show the segmentation results for the same image but different networks so the differences can be compared. Briefly describe the results you obtain and any observations. **[10 points]** 
    6. Explain your observations in terms of: (i) what choices helped improve the accuracy and (ii) what other steps could have been tried to further improve accuracy?  **[10 points]**

``Answer Q1.4.A here``

The cross-entropy loss is used to train the semantic segmentation network. For each pixel, the prediction (weights/distribution over 21 classes) is normalized by the softmax function, and then use the ground truth label/distribution to compute the cross-entropy. Finally the loss is the average loss among all pixels.

``Answer Q1.4.B here``

The Adam optimizer is used in our training. The initial learning rate is set to $0.0001$ and decrese $10\%$ every $20$ epoches. The batchsize is $16$ and batch normalization is applied after every convolution. We trained the network for $120$ epoches, and the loss decreased. However, we were not able to train the network to converge due to limited computation time. To prevent/alleviate overfitting, the dataloader is set to shuffle batches.

``Answer Q1.4.C here``

<img src='./Segmentation/Q1-4-C.png' width='400px' style='display: inline-block;'/>

``Answer Q1.4.D here``


| class | unet | +dilation | +dilation + spp |
|:------|:-----|:----------|:----------------|
| 0 | 86.684425 | 87.9268 | 86.97555 |
| 1 | 57.10929 | 59.55688 | 51.525707 |
| 2 | 22.867416 | 33.38484 | 25.032478 |
| 3 | 29.204008 | 33.917114 | 38.85373 |
| 4 | 34.73051 | 22.035831 | 29.414698 |
| 5 | 28.292776 | 22.783867 | 34.48598 |
| 6 | 55.199417 | 55.937737 | 62.64902 |
| 7 | 51.81139 | 53.432972 | 60.39775 |
| 8 | 45.028877 | 52.3381 | 50.46089 |
| 9 | 10.747867 | 12.301113 | 11.91936 |
| 10 | 21.77799 | 28.283752 | 33.716442 |
| 11 | 27.676928 | 28.40639 | 29.298407 |
| 12 | 30.588356 | 40.389996 | 35.617523 |
| 13 | 22.19011 | 34.82432 | 37.583836 |
| 14 | 40.525784 | 34.445736 | 47.564754 |
| 15 | 60.0953 | 63.4203 | 62.921764 |
| 16 | 24.400042 | 30.901745 | 16.521242 |
| 17 | 36.724792 | 44.46482 | 45.577274 |
| 18 | 20.487953 | 17.529568 | 20.850365 |
| 19 | 47.448513 | 42.49484 | 52.157803 |
| 20 | 36.34981 | 36.480244 | 40.063618 |
| mIoU | 37.616264 | 39.77414 | 41.599434 |

``Answer Q1.4.E here``

From the mIoU in Q1.4.D, we can quantitatively know that both dilation and SPP help capture more global context. `+dilation` gives better result than that of `unet`, and `+dilation+spp` further improves the segmentation.

From the visual results below, we can draw the same conclusion in some cases. Take the first image for example, the airplane is missed by `unet`, roughly detected by `+dilation`, and more accurately detected by `+dilation+spp`.

However, in some situation, the SPP yields worse result. Take the second (a dog on the sofa) and the third (a car) image for example, the SPP introduces more errors.

Overall, the `+dilation` makes stable improvement, and the improvement of `+dilation+spp` is not as stable as that of `+dilation`.

<img src='./Segmentation/save_0.png' width='400px' style='display: inline-block;'/>
<img src='./Segmentation/save_1.png' width='400px' style='display: inline-block;'/>

<img src='./Segmentation/save_2.png' width='400px' style='display: inline-block;'/>
<img src='./Segmentation/save_3.png' width='400px' style='display: inline-block;'/>

``Answer Q1.4.F here``

From the mIoU in Q1.4.D, we can quantitatively know that both dilation and SPP help capture more global context. `+dilation` gives better result than that of `unet`, and `+dilation+spp` further improves the segmentation. The dilation incorporates more global context by enlarging the receptive field. The SPP incorporates global context of different scales and different postions.

During the training process, we learned that the learning rate is critical to improve accuracy. Initially the learning rate was set to $0.001$ and the network basically learned nothing, with training accuracy of less than $30\%$. After lowering the learning rate to $0.0001$, the network learned more efficiently and achieved an accuracy of $60\%$.

In terms of future work, one thing we can do is to train the network until it converges. Currently, the network does not completely converge but still achieve a reasonable visual result. Also we can augment the training images. For example, we can simulate different lighting condition by changing the image intensity, so that the network can handle lighting variance. Furthering fine tune the learning is important, adatively changing the learning based on the validation accuracy.

## Q2: SSD [3] Object Detection

1. **Check the codes in `Detection`.** The codes are modified from ``https://github.com/amdegroot/ssd.pytorch``. Run `eval.py` code to get the object detection average precision (AP) on the PASCAL VOC 2012 dataset. The model is already trained on the PASCAL VOC 2012 object detection dataset and stored at ``/datasets/cs252-sp21-A00-public/hw3_data/detection/weights/VOC.pth``. Draw a table in your report summarizing the AP of all 20 object categories and their mean.   **[10 points]**

``Answer Q2.1 here``

| class | AP |
|:------|:---|
| aeroplane | 0.7818 |
| bicycle | 0.7200 |
| bird | 0.6477 |
| boat | 0.4651 |
| bottle | 0.3866 |
| bus | 0.7714 |
| car | 0.6852 |
| cat | 0.8264 |
| chair | 0.4571 |
| cow | 0.6483 |
| diningtable | 0.5567 |
| dog | 0.7803 |
| horse | 0.7255 |
| motorbike | 0.7716 |
| person | 0.7421 |
| pottedplant | 0.3492 |
| sheep | 0.6679 |
| sofa | 0.5870 |
| train | 0.7815 |
| tvmonitor | 0.6457 |
| Mean AP | 0.6499 |

2. **Answer the following questions:**
    1.  Briefly explain how average precision is computed for PASCAL VOC 2012 dataset. Please check the code ($\mathtt{eval.py:~Line~163-191}$). In this homework, we use the Pascal VOC 2007 metric. **[10 points]** 
    2. Explain how SSD can be much faster compared to Faster RCNN [4]? **[10 points]**
    3. Usually the number of negative bounding boxes (boxes without any object) is much larger than the number of positive bounding boxes. Explain how this imbalance is handled in SSD and Faster RCNN, respectively. **[10 points]**

``Answer Q2.2.A here``

The predicted bounding boxes are evaluated from the most confidence to the least confidence. The predicted bounding box is considered true positive, if its largest IoU across all ground truth bounding boxes, is larger than the threshold (0.5). The ground truth bounding box with largest IoU is marked as dectected and cannot be detected again. Otherwise the predicted bounding is a false positive.

`precision`: TP / (TP + FP)  
`recall`: TP / (TP + FN)  
`average precision`: The average of maximum precision at different recall levels (0.0, 0.1, 0.2, ..., 1.0).

``Answer Q2.2.B here``

SSD is a one-shot solution for objection detection. Instead of two stages of region proposal and Fast R-CNN object detection, SSD combines the object detection and bounding box regression in one convolution operation, and thus it is faster.

``Answer Q2.2.C here`` 

"Instead of using all the negative examples, SSD sorts them by the highest confidence loss for each predicted default box and pick the top ones so that the ratio between the negatives and positives is at most 3:1" [3]


3. Randomly pick up some images from the PASCAL VOC 2012 dataset and some from other sources. Visualize the bounding box prediction results and include a figure in your report. You can use the code in folder $\mathtt{demo}$ for visualization. **[10 points]**

``Answer Q2.3 here`` 

The first image is picked from PASCAL VOC 2012, the boat is well recognized. The second and third are images from me. We can the network is trying to detect the warship as a boat but fail to bound the whole warship. The network is good at detecting people, even from the back.

<img src='./image-1.png' width='400px' style='display: inline-block;'/>  

<img src='./image-2.png' width='400px' style='display: inline-block;'/>  

<img src='./image-3.png' width='400px' style='display: inline-block;'/>

## References
1. Yu, Fisher, and Vladlen Koltun. "Multi-scale context aggregation by dilated convolutions." arXiv preprint arXiv:1511.07122 (2015).
2. Zhao, Hengshuang, et al. "Pyramid scene parsing network." Proceedings of the IEEE conference on computer vision and pattern recognition. 2017.
3. Liu, Wei, et al. "Ssd: Single shot multibox detector." European conference on computer vision. Springer, Cham, 2016.
4.  Ren, Shaoqing, et al. "Faster r-cnn: Towards real-time object detection with region proposal networks." Advances in neural information processing systems. 2015.